In [79]:
from scipy.stats import skew, kurtosis, norm
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.decomposition import FastICA
import cv2
import requests
from io import BytesIO
import random
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
from sklearn.decomposition import FastICA
import warnings
import cvxpy as cp
from scipy.optimize import linprog
import time
import seaborn as sns
import os
from PIL import Image
from scipy.stats import pearsonr
from scipy.optimize import minimize_scalar
from scipy.ndimage import maximum_filter
import math

In [395]:
def unit_sphere(angles):
    #print(angles)
    angles = np.array(angles)
    d = len(angles) + 1
    coords = np.zeros(d)
    prod = 1.0
    # For the first d-1 coordinates
    for i in range(d - 1):
        coords[i] = prod * np.cos(angles[i])
        prod *= np.sin(angles[i])
    # The final coordinate
    coords[-1] = prod
    return coords

def unit_matrix(matrix, thetas):
    new_matrix = []
    for row in range(len(matrix)):
        new_matrix.append(np.sum((matrix.T * unit_sphere(thetas[row])).T, axis=0))
    return np.array(new_matrix)

def get_midpoint(matrix):
    pca = PCA(n_components=matrix.shape[0])
    pca.fit(matrix.T)
    return math.acos(pca.components_[0,0]) % np.pi

def intersection_objective(difference, midpoint, matrix):
    thetas = np.array([[midpoint - difference] * (matrix.shape[0] - 1)] * matrix.shape[0])
    thetas[1:matrix.shape[0], :] = thetas[1:matrix.shape[0],:]  + np.eye(matrix.shape[0] - 1) * 2 * difference
    return np.mean(abs(np.corrcoef(unit_matrix(matrix, thetas)) - np.eye(matrix.shape[0])))
              
def gaussian_seperator(matrix):
    midpoint = get_midpoint(matrix)
    res = minimize_scalar(
            lambda diff: intersection_objective(diff, midpoint, matrix),
            bounds=(-np.pi, np.pi),
            method='bounded'
    )

    difference = res.x
    thetas = np.array([[midpoint - difference] * (matrix.shape[0] - 1)] * matrix.shape[0])
    thetas[1:matrix.shape[0], :] = thetas[1:matrix.shape[0],:]  + np.eye(matrix.shape[0] - 1) * 2 * difference
    print(res)
    return unit_matrix(matrix, thetas)
    


In [397]:
matrix = a
midpoint = 0.5
difference = 0.1
#thetas = np.array([[midpoint - difference] * (matrix.shape[0] - 1)] * matrix.shape[0]
#thetas[:, :(matrix.shape[0] - 2)]

In [438]:
pd.options.display.float_format = '{:.3f}'.format


theta1 = np.random.uniform(0, np.pi)
theta2 = np.random.uniform(0, np.pi)
mixing_matrix = np.array([
    [np.cos(theta1), np.sin(theta1)],
    [np.cos(theta2), np.sin(theta2)]
])

normal_matrix = np.vstack((np.random.normal(loc=0, scale=1, size=50), np.random.normal(loc=0, scale=1, size=50), np.random.normal(loc=0, scale=1, size=50)))
normal_matrix = np.vstack((np.random.normal(loc=0, scale=1, size=50), np.random.normal(loc=0, scale=1, size=50)))
mixing_matrix = np.random.uniform(0.5, 1, (len(normal_matrix), len(normal_matrix)))  # Random weights
mixed_normal = mixing_matrix @ normal_matrix  # Linear combination
results = gaussian_seperator(mixed_normal)
pd.DataFrame(np.corrcoef(normal_matrix, results))

 message: Solution found.
 success: True
  status: 0
     fun: 1.4517089428572095e-07
       x: 1.7535181555055677
     nit: 23
    nfev: 23


,0,1,2,3
0,1.000,-0.038,-0.088,-0.996
1,-0.038,1.000,-0.992,0.125
2,-0.088,-0.992,1.000,-0.000
3,-0.996,0.125,-0.000,1.000


In [403]:
np.array(thetas)[:, :]

array([[0.4, 0.4, 0.4],
       [0.4, 0.4, 0.4],
       [0.4, 0.4, 0.4],
       [0.4, 0.4, 0.4]])

In [288]:
def gaussian_seperator(matrix):
    """
    Given a 2 x N matrix, find a rotation matrix that minimizes the
    correlation (i.e. makes the two rows as orthogonal as possible).
    
    Returns the optimal 2x2 rotation matrix and the optimal rotation angle.
    """
    midpoint = get_midpoint(matrix)
    # Search for a rotation angle in a window around the midpoint.
    # (Here we use midpoint ± pi/4 as an example search interval.)
    search_lower = midpoint - np.pi/4
    search_upper = midpoint + np.pi/4

    res = opt.minimize_scalar(lambda theta: intersection_objective(theta, matrix),
                              bounds=(search_lower, search_upper),
                              method='bounded')
    optimal_theta = res.x
    U_opt = rotation_matrix(optimal_theta)
    return U_opt, optimal_theta

In [ ]:
matrix_4x10 = 
print("Random 4x10 matrix:")
print(matrix_4x10)

# Generate a random 3x4 matrix (3 rows, 4 columns)
matrix_3x4 = np.random.randn(3, 4)

<function __main__.unit_matrix(matrix, thetas)>